<a href="https://colab.research.google.com/github/RIDDHI1624/Drug-Discovery/blob/main/Energyminimization_proj_allo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install gromacs -y -qq

Preconfiguring packages ...
Selecting previously unselected package sse4.2-support.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../0-sse4.2-support_6_amd64.deb ...
Unpacking sse4.2-support (6) ...
Selecting previously unselected package fonts-mathjax.
Preparing to unpack .../1-fonts-mathjax_2.7.9+dfsg-1_all.deb ...
Unpacking fonts-mathjax (2.7.9+dfsg-1) ...
Selecting previously unselected package libfftw3-double3:amd64.
Preparing to unpack .../2-libfftw3-double3_3.3.8-2ubuntu8_amd64.deb ...
Unpacking libfftw3-double3:amd64 (3.3.8-2ubuntu8) ...
Selecting previously unselected package libfftw3-single3:amd64.
Preparing to unpack .../3-libfftw3-single3_3.3.8-2ubuntu8_amd64.deb ...
Unpacking libfftw3-single3:amd64 (3.3.8-2ubuntu8) ...
Selecting previously unselected package libgromacs6:amd64.
Preparing to unpack .../4-libgromacs6_2021.4-2_amd64.deb ...
Unpacking libgromacs6:amd64 (2021.4-2) ...
Selecting previously unselected package libjs-m

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/allostery
%cd /content/allostery

Mounted at /content/drive
/content/allostery


In [3]:
from google.colab import files
uploaded = files.upload()  # Click "Choose Files" and select alphafold_champion.pdb

Saving alphafold_champion.pdb to alphafold_champion.pdb


In [4]:
!grep "^ATOM" alphafold_champion.pdb > protein_clean.pdb
!echo "Number of atoms:"
!wc -l protein_clean.pdb

Number of atoms:
2071 protein_clean.pdb


In [9]:
!echo "1" | gmx pdb2gmx -f protein_clean.pdb 2>&1 | grep -E "^\s+[0-9]+"

^C


In [10]:
# Change the number if AMBER99SB-ILDN is not 6 in your list
!echo "6" | gmx pdb2gmx -f protein_clean.pdb -o protein_processed.gro -water tip3p -ignh

             :-) GROMACS - gmx pdb2gmx, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

In [11]:
!gmx editconf -f protein_processed.gro -o protein_box.gro -c -d 1.0 -bt dodecahedron

             :-) GROMACS - gmx editconf, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff           

In [12]:
!gmx solvate -cp protein_box.gro -cs spc216.gro -o protein_solv.gro -p topol.top

             :-) GROMACS - gmx solvate, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

In [13]:
%%writefile ions.mdp
integrator  = steep
emtol       = 1000.0
emstep      = 0.01
nsteps      = 50000
nstlist     = 1
cutoff-scheme = Verlet
ns_type     = grid
coulombtype = PME
rcoulomb    = 1.0
rvdw        = 1.0
pbc         = xyz

Writing ions.mdp


In [14]:
!gmx grompp -f ions.mdp -c protein_solv.gro -p topol.top -o ions.tpr -maxwarn 1
!echo "SOL" | gmx genion -s ions.tpr -o protein_ions.gro -p topol.top -pname NA -nname CL -neutral -conc 0.15

              :-) GROMACS - gmx grompp, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

In [15]:
%%writefile em.mdp
integrator  = steep
emtol       = 1000.0
emstep      = 0.01
nsteps      = 5000
nstlist     = 1
cutoff-scheme = Verlet
ns_type     = grid
coulombtype = PME
pme_order   = 4
fourierspacing = 0.16
rcoulomb    = 1.0
rvdw        = 1.0
pbc         = xyz
DispCorr    = EnerPres
nstenergy   = 500
nstlog      = 500

Writing em.mdp


In [16]:
!gmx grompp -f em.mdp -c protein_ions.gro -p topol.top -o em.tpr

              :-) GROMACS - gmx grompp, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

In [17]:
!gmx mdrun -v -deffnm em

              :-) GROMACS - gmx mdrun, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            T

In [18]:
!echo "10" | gmx energy -f em.edr -o potential.xvg 2>&1 | tail -20

  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Pres.-DC      
 13  Pressure        14  Vir-XX          15  Vir-XY          16  Vir-XZ        
 17  Vir-YX          18  Vir-YY          19  Vir-YZ          20  Vir-ZX        
 21  Vir-ZY          22  Vir-ZZ          23  Pres-XX         24  Pres-XY       
 25  Pres-XZ         26  Pres-YX         27  Pres-YY         28  Pres-YZ       
 29  Pres-ZX         30  Pres-ZY         31  Pres-ZZ         32  #Surf*SurfTen 
 33  T-rest        

Last energy frame read 391 time  494.000         

GROMACS reminds you: "Sisters Have Always Fascinated Me" (Speech)


Statistics over 495 steps [ 0.0000 through 494.0000 ps ], 1 data sets
All statistics are over 392 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Coul. recip.                8143.52 

In [19]:
!echo -e "1\n1" | gmx trjconv -s em.tpr -f em.gro -o alphafold_champion_minimized.pdb -pbc mol -center

             :-) GROMACS - gmx trjconv, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

In [20]:
!cp alphafold_champion_minimized.pdb /content/drive/MyDrive/
from google.colab import files
files.download("alphafold_champion_minimized.pdb")
print("Energy minimization complete! Ready for LiteFold Rosalind.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Energy minimization complete! Ready for LiteFold Rosalind.
